In [1]:
import pandas as pd
import numpy as np

In [5]:
dataset = pd.read_excel('teste.xlsx')
dataset

,Grupo,EST_Codigo,EST_nome,status,dt_pedido,dt_vencimento,path,cond_pag,cod_ped,COD_cpg,sequencial,cont_avuls,form_pag,nota,setor,tipo,tipo_pedido
0,CONDONAL,9,CONDONAL COMERCIO E SERVICOS LTDA,APROVADO,2025-02-18,2025-02-21,Y:\CELULA FISCAL - FORTES\CONDONAL\ROMERO_Admi...,21/02/2025,20250200038,20250200916,1.0,NAO,BOLETO,SIM,ADMINISTRATIVO,PAGAMENTO,PEDIDO COM NOTA
1,CONDONAL,8,CONDONAL ADMINISTRACAO DE CONDOMINIOS LTDA (RPJ),APROVADO,2025-02-18,NaT,Y:\CELULA FISCAL - FORTES\CONDONAL\GESTÃO DE C...,11/04/2025,20250200051,NaN,NaN,SIM,BOLETO,NÃO,GESTÃO DE CONDOMÍNIO,PAGAMENTO,PEDIDO AVULSO
2,CONDONAL,3,CONDO SERVICOS EM CONDOMINIOS LTDA,APROVADO,2025-02-21,2025-02-27,Y:\CELULA FISCAL - FORTES\CONDONAL\DP\SWILE - ...,27/02/2025,20250200124,20250201015,1.0,NAO,BOLETO,SIM,LMO,PAGAMENTO,PEDIDO COM NOTA
3,CONDONAL,3,CONDO SERVICOS EM CONDOMINIOS LTDA,APROVADO,2025-02-21,2025-02-27,Y:\CELULA FISCAL - FORTES\CONDONAL\DP\SWILE - ...,27/02/2025,20250200124,20250201015,1.0,NAO,BOLETO,SIM,LMO,PAGAMENTO,PEDIDO COM NOTA
4,CONDONAL,3,CONDO SERVICOS EM CONDOMINIOS LTDA,APROVADO,2025-02-21,2025-02-27,Y:\CELULA FISCAL - FORTES\CONDONAL\DP\SWILE - ...,27/02/2025,20250200124,20250201015,1.0,NAO,BOLETO,SIM,LMO,PAGAMENTO,PEDIDO COM NOTA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37156,FOLK,2,FOLK PORTARIA REMOTA LTDA,APROVADO,2025-04-30,2025-05-07,Z:\CELULA FISCAL - FORTES\FOLK\LUCAS\07 05 202...,BOLETO 07 05 2025,20250400540,20250500472,1.0,NÃO,BOLETO,SIM,OPERACIONAL,PAGAMENTO,PEDIDO COM NOTA
37157,FOLK,2,FOLK PORTARIA REMOTA LTDA,APROVADO,2025-05-01,2025-05-07,Z:\CELULA FISCAL - FORTES\FOLK\LUCAS\07 05 202...,BOLETO 07 05 2025,20250500052,20250500474,1.0,NÃO,BOLETO,SIM,OPERACIONAL,PAGAMENTO,PEDIDO COM NOTA
37158,FOLK,2,FOLK PORTARIA REMOTA LTDA,APROVADO,2025-05-01,2025-05-07,Z:\CELULA FISCAL - FORTES\FOLK\LUCAS\07 05 202...,BOLETO 07 05 2025,20250500052,20250500474,1.0,NÃO,BOLETO,SIM,OPERACIONAL,PAGAMENTO,PEDIDO COM NOTA
37159,FOLK,2,FOLK PORTARIA REMOTA LTDA,APROVADO,2025-05-07,2025-05-15,Y:\CELULA FISCAL - FORTES\FOLK\LUCAS\15 05 202...,BOLETO 15 05 2025,20250500185,20250500769,1.0,NÃO,BOLETO,SIM,T.I.,PAGAMENTO,PEDIDO COM NOTA


In [11]:
dataset.iloc[[290]]['path'].isna()

290    True
Name: path, dtype: bool

In [19]:
subset.iloc[[4]]

,Grupo,EST_Codigo,EST_nome,status,dt_pedido,dt_vencimento,path,cond_pag,cod_ped,COD_cpg,sequencial,cont_avuls,form_pag,nota,setor,tipo,tipo_pedido
291,CONDONAL,7,CONDO SERVICOS DE APOIO ADMINISTRATIVO LTDA (P&R),REJEITADO,2025-04-01,NaT,Y:\CELULA FISCAL - FORTES\CONDONAL\Renata\NF 2...,04/04/2025,20250400025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PEDIDO AVULSO


In [24]:
subset = dataset[287:292]

In [27]:
subset.iloc[[1]]['path'].isna()

288    False
Name: path, dtype: bool

In [28]:
subset_path = []
subset_path.append(subset.iloc[[i]])

In [45]:
# subset.iloc[[3]]['path'].isna()
subset.iloc[[3]]['path'].isna()

290    True
Name: path, dtype: bool

In [54]:
subset.iloc[[1]]['path']

288    Y:\CELULA FISCAL - FORTES\CONDONAL\OPERACIONAL...
Name: path, dtype: object

In [49]:
subset = dataset[287:292]

subset_path = []
for i in range(0, 5):
    if not subset.iloc[[i]]['path'].empty:
        subset_path.append(subset.iloc[[i]])
        # continue
    # else:
    #     subset_path.append(subset.iloc[[i]])
    # print(subset[i])

In [51]:
subset_path

[        Grupo  EST_Codigo                                EST_nome    status  \
 287  CONDONAL          10  CONDONAL SERVICOS E ADMINISTRACAO LTDA  APROVADO   
 
      dt_pedido dt_vencimento  \
 287 2025-04-03    2025-04-11   
 
                                                   path    cond_pag  \
 287  Y:\CELULA FISCAL - FORTES\CONDONAL\OPERACIONAL...  11/04/2025   
 
          cod_ped      COD_cpg  sequencial cont_avuls form_pag nota setor  \
 287  20250400092  20250400656         1.0        NÃO      PIX  SIM   LMO   
 
           tipo      tipo_pedido  
 287  PAGAMENTO  PEDIDO COM NOTA  ,
         Grupo  EST_Codigo                                EST_nome    status  \
 288  CONDONAL          10  CONDONAL SERVICOS E ADMINISTRACAO LTDA  APROVADO   
 
      dt_pedido dt_vencimento  \
 288 2025-04-03    2025-04-11   
 
                                                   path    cond_pag  \
 288  Y:\CELULA FISCAL - FORTES\CONDONAL\OPERACIONAL...  11/04/2025   
 
          cod_ped      CO

In [14]:
subset

,Grupo,EST_Codigo,EST_nome,status,dt_pedido,dt_vencimento,path,cond_pag,cod_ped,COD_cpg,sequencial,cont_avuls,form_pag,nota,setor,tipo,tipo_pedido
287,CONDONAL,10,CONDONAL SERVICOS E ADMINISTRACAO LTDA,APROVADO,2025-04-03,2025-04-11,Y:\CELULA FISCAL - FORTES\CONDONAL\OPERACIONAL...,11/04/2025,20250400092,20250400656,1.0,NÃO,PIX,SIM,LMO,PAGAMENTO,PEDIDO COM NOTA
288,CONDONAL,10,CONDONAL SERVICOS E ADMINISTRACAO LTDA,APROVADO,2025-04-03,2025-04-11,Y:\CELULA FISCAL - FORTES\CONDONAL\OPERACIONAL...,11/04/2025,20250400092,20250400656,1.0,NÃO,PIX,SIM,LMO,PAGAMENTO,PEDIDO COM NOTA
289,CONDONAL,10,CONDONAL SERVICOS E ADMINISTRACAO LTDA,APROVADO,2025-04-04,2025-04-14,Y:\CELULA FISCAL - FORTES\CONDONAL\OPERACIONAL...,14/04/2025,20250400116,20250400671,1.0,NÃO,PIX,SIM,LMO,PAGAMENTO,PEDIDO COM NOTA
290,CONDONAL,7,CONDO SERVICOS DE APOIO ADMINISTRATIVO LTDA (P&R),REJEITADO,2025-02-17,NaT,NaN,24/02/2025,20250200021,NaN,NaN,SIM,BOLETO,NÃO,GETSÃO DE CONDOMÍNIO,PAGAMENTO,PEDIDO AVULSO
291,CONDONAL,7,CONDO SERVICOS DE APOIO ADMINISTRATIVO LTDA (P&R),REJEITADO,2025-04-01,NaT,Y:\CELULA FISCAL - FORTES\CONDONAL\Renata\NF 2...,04/04/2025,20250400025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PEDIDO AVULSO


In [104]:
from datetime import datetime

In [62]:
data = datetime(25, 6, 30)
data

datetime.datetime(25, 6, 30, 0, 0)

In [108]:
data = "10/03/2025  00:00:00"
data = datetime.strftime(data)

TypeError: descriptor 'strftime' for 'datetime.date' objects doesn't apply to a 'str' object

In [87]:
import re

In [95]:
data_venc = 'BOLETO VENC. 27/06/2025'

In [97]:
datas = re.search(r'\b(?:0[1-9]|[12][0-9]|3[01])/(?:0[1-9]|1[0-2])/\d{4}\b', data_venc)
print(datas.group())  # ['30/06/2025', '01/07/2025']

27/06/2025


In [99]:
type(datas.group())

str